<a href="https://colab.research.google.com/github/sode-k/sode-k/blob/main/Rec2txt_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install resemblyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 106.8 MB/s eta 0:00:00
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73506 sha256=68699b25bc37ebe667ee5fb1586b9d3975edad3104a17749b81a98e3f3f62432
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee2f
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26303 sha256=528642a1b169e7a7f50dd1ed154f4da430c9d8c810af1253cb099138706823dc
  Stored in directory: /root/.cache/pip/wheels/9d/67/2f/53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built webrtcvad typing


In [ ]:
import torch
from transformers import pipeline
import librosa
import soundfile as sf
from resemblyzer import VoiceEncoder, preprocess_wav
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm
import numpy as np
import os
from google.colab import drive, files

drive.mount('/content/drive')

model_name = "kotoba-tech/kotoba-whisper-v2.2"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    transcribe = pipeline("automatic-speech-recognition", model=model_name, device=0 if device=="cuda" else -1)
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading model:", e)
    exit()

audio_file = "/content/drive/MyDrive/奨学金問題/2025年1月23日・増田先生.m4a"

if not os.path.exists(audio_file):
    print(f"Error: File '{audio_file}' does not exist.")
    exit()

try:
    y, sr = librosa.load(audio_file, sr=16000)
    sf.write("processed_audio.wav", y, sr)
    print(f"Audio file '{audio_file}' loaded and converted to WAV format.")
except Exception as e:
    print("Error processing audio file:", e)
    exit()

try:
    encoder = VoiceEncoder(device=device)
    print("Voice encoder initialized.")
except Exception as e:
    print("Error initializing voice encoder:", e)
    exit()

try:
    segments = librosa.effects.split(y, top_db=20)
    num_segments = len(segments)
    print(f"Audio split into {num_segments} segments.")
except Exception as e:
    print("Error splitting audio:", e)
    exit()

embeddings_list = []
for segment in tqdm(segments, desc="Extracting embeddings"):
    start, end = segment
    segment_y = y[start:end]
    sf.write("temp.wav", segment_y, sr)
    try:
        wav_segment = preprocess_wav("temp.wav")
        embedding = encoder.embed_utterance(wav_segment)
        embeddings_list.append(embedding)
    except Exception as e:
        print("Error embedding segment:", e)
        embeddings_list.append(np.zeros(encoder.embedding_size))
os.remove("temp.wav")

try:
    clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.0)
    cluster_labels = clustering.fit_predict(embeddings_list)
    print("Clustering completed.")
except Exception as e:
    print("Error during clustering:", e)
    exit()

try:
    df = []
    for i, segment in enumerate(segments):
        start_time = segment[0] / sr
        end_time = segment[1] / sr
        df.append({'start': start_time, 'end': end_time, 'label': cluster_labels[i]})
    print("Segments labeled successfully.")
except Exception as e:
    print("Error labeling segments:", e)
    exit()

results = []
for item in tqdm(df, desc="Transcribing segments"):
    start_sample = int(item['start'] * sr)
    end_sample = int(item['end'] * sr)
    segment_y = y[start_sample:end_sample]
    sf.write("temp_segment.wav", segment_y, sr)
    try:
        transcription = transcribe("temp_segment.wav")
        if isinstance(transcription, dict):
            text = transcription.get('text', '').strip()
        elif isinstance(transcription, list) and len(transcription) > 0:
            text = transcription[0].get('text', '').strip()
        else:
            text = ''
        speaker = f"Speaker {item['label']}"
        if text:
            results.append(f"{speaker}: {text}")
    except Exception as e:
        print(f"Error transcribing segment {item['label']}: {e}")
    os.remove("temp_segment.wav")

if results:
    chat_transcript = "\n".join(results)
    try:
        with open("chat_transcript.txt", "w", encoding="utf-8") as f:
            f.write(chat_transcript)
        print("Transcript saved to 'chat_transcript.txt'.")
        files.download("chat_transcript.txt")
    except Exception as e:
        print("Error saving transcript:", e)
else:
    print("No transcription results to save.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

Clustering completed.
Segments labeled successfully.


Streaming output truncated to the last 5000 lines.
  warnings.warn(
Transcribing segments:  11%|█         | 294/2793 [02:08<17:28,  2.38it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Transcribing segments:  11%|█         | 295/2793 [02:08<17:26,  2.39it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Transcribing segments:  11%|█         | 296/2793 [02:08<17:24,  2.39it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Transcribing segments:  11%|█         | 297/2793 [02:09<17:24,  2

Transcript saved to 'chat_transcript.txt'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>